# 1. Import libraries

In [5]:
!! pip install geocoder

['Collecting geocoder',
 '  Downloading https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl (98kB)',
 'Requirement already satisfied: requests in /opt/conda/envs/Python36/lib/python3.6/site-packages (from geocoder) (2.21.0)',
 'Collecting ratelim (from geocoder)',
 '  Downloading https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl',
 'Requirement already satisfied: six in /opt/conda/envs/Python36/lib/python3.6/site-packages (from geocoder) (1.12.0)',
 'Requirement already satisfied: click in /opt/conda/envs/Python36/lib/python3.6/site-packages (from geocoder) (7.0)',
 'Requirement already satisfied: future in /opt/conda/envs/Python36/lib/python3.6/site-packages (from geocoder) (0.17.1)',
 'Requirement already satisfied: urllib3<1.25,>=1.21.1 in /opt/conda/envs/Python36/lib/python3.6/site-packages (from requ

In [7]:
!! pip install folium

['Collecting folium',
 '  Downloading https://files.pythonhosted.org/packages/72/ff/004bfe344150a064e558cb2aedeaa02ecbf75e60e148a55a9198f0c41765/folium-0.10.0-py2.py3-none-any.whl (91kB)',
 'Collecting branca>=0.3.0 (from folium)',
 '  Downloading https://files.pythonhosted.org/packages/63/36/1c93318e9653f4e414a2e0c3b98fc898b4970e939afeedeee6075dd3b703/branca-0.3.1-py3-none-any.whl',
 'Requirement already satisfied: requests in /opt/conda/envs/Python36/lib/python3.6/site-packages (from folium) (2.21.0)',
 'Requirement already satisfied: numpy in /opt/conda/envs/Python36/lib/python3.6/site-packages (from folium) (1.15.4)',
 'Requirement already satisfied: jinja2>=2.9 in /opt/conda/envs/Python36/lib/python3.6/site-packages (from folium) (2.10)',
 'Requirement already satisfied: six in /opt/conda/envs/Python36/lib/python3.6/site-packages (from branca>=0.3.0->folium) (1.12.0)',
 'Requirement already satisfied: certifi>=2017.4.17 in /opt/conda/envs/Python36/lib/python3.6/site-packages (from

In [8]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values # to get coordinates

import requests
import geocoder# library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries  are imported.")

Libraries  are imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [9]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [11]:
# create a list to store neighborhood data
neighborhoodList = []

In [12]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [13]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 4 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [14]:
kl_df.shape

(58, 1)

# 3. Get the geographical coordinates

In [15]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [18]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [19]:
coords

[[12.841257338311804, 77.48156456758687],
 [12.997981959201338, 77.61048458742573],
 [12.93949852510253, 77.54449832653358],
 [12.997818984736565, 77.61040562017429],
 [12.939010000000053, 77.57136000000008],
 [12.882480000000044, 77.62473000000006],
 [12.927350000000047, 77.67184000000003],
 [12.997981959201338, 77.61048458742573],
 [12.966850000000022, 77.58742000000007],
 [12.957985739107851, 77.600930621011],
 [12.817540000000065, 77.67879000000005],
 [12.966272502719853, 77.606767849728],
 [12.855505840960276, 77.73109527298159],
 [13.249930000000063, 77.70792000000006],
 [12.99785498473656, 77.6104270618905],
 [12.943300000000022, 77.65603000000004],
 [12.839877785463049, 77.67227766209761],
 [12.998850000000061, 77.61271000000005],
 [12.942780000000027, 77.54121000000004],
 [13.047470266720541, 77.63328243075193],
 [13.049690000000055, 77.58951000000008],
 [12.957454844018699, 77.60090848251485],
 [12.912110000000041, 77.64490000000006],
 [12.978230000000053, 77.63429000000008],

In [20]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [21]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [22]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(58, 3)


,Neighborhood,Latitude,Longitude
0,"► Agara, Bangalore‎ (2 C, 6 F)",12.841257,77.481565
1,► Arekere‎ (5 F),12.997982,77.610485
2,"► Banashankari‎ (1 C, 4 F)",12.939499,77.544498
3,► Banaswadi‎ (2 F),12.997819,77.610406
4,"► Basavanagudi‎ (5 C, 11 F)",12.939010,77.571360
5,"► Begur, Bangalore‎ (1 C, 6 F)",12.882480,77.624730
6,"► Bellandur‎ (1 C, 4 F)",12.927350,77.671840
7,► BEML‎ (5 F),12.997982,77.610485
8,"► Bengaluru Pete‎ (9 C, 4 F)",12.966850,77.587420
9,"► Bidadi‎ (2 C, 1 F)",12.957986,77.600931


In [23]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

# 4. Create a map of Bangalore with neighborhoods superimposed on top

In [24]:
# get the coordinates of Bangalore
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [25]:
# create map of Bangalore using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [26]:
# save the map as HTML file
map_kl.save('map_kl.html')

# 5. Use the Foursquare API to explore the neighborhoods¶

In [27]:
# define Foursquare Credentials and Version
CLIENT_ID = 'KRY1OQXPKKNAZLOLHZ1X5RCA4U3SLELIAQME0UJFZOAGY4SC' # your Foursquare ID
CLIENT_SECRET = 'HI0OIED01R1TYK0Q2B0II35WL1SABPOFB3UZNCFRIMFMRG3Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KRY1OQXPKKNAZLOLHZ1X5RCA4U3SLELIAQME0UJFZOAGY4SC
CLIENT_SECRET:HI0OIED01R1TYK0Q2B0II35WL1SABPOFB3UZNCFRIMFMRG3Q


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3820, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,► Arekere‎ (5 F),12.997982,77.610485,Shahi Kabab,12.995927,77.613525,BBQ Joint
1,► Arekere‎ (5 F),12.997982,77.610485,Mangalore Pearl - Seafood Restaurant,12.994472,77.615551,Seafood Restaurant
2,► Arekere‎ (5 F),12.997982,77.610485,Naturals Icecream,12.996912,77.611268,Ice Cream Shop
3,► Arekere‎ (5 F),12.997982,77.610485,Lassi Shop Downtown,13.000869,77.613731,Juice Bar
4,► Arekere‎ (5 F),12.997982,77.610485,Richies - Frazer Town,13.000930,77.613150,Indian Restaurant


Let's check how many venues were returned for each neighorhood

In [30]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
► Arekere‎ (5 F),100,100,100,100,100,100
► BEML‎ (5 F),100,100,100,100,100,100
"► Banashankari‎ (1 C, 4 F)",26,26,26,26,26,26
► Banaswadi‎ (2 F),100,100,100,100,100,100
"► Basavanagudi‎ (5 C, 11 F)",100,100,100,100,100,100
"► Begur, Bangalore‎ (1 C, 6 F)",14,14,14,14,14,14
"► Bellandur‎ (1 C, 4 F)",89,89,89,89,89,89
"► Bengaluru Pete‎ (9 C, 4 F)",100,100,100,100,100,100
"► Bidadi‎ (2 C, 1 F)",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


In [32]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['BBQ Joint', 'Seafood Restaurant', 'Ice Cream Shop', 'Juice Bar',
       'Indian Restaurant', 'Park', 'Department Store',
       'Fast Food Restaurant', 'Pakistani Restaurant',
       'Middle Eastern Restaurant', 'Pub', 'Snack Place', 'Pizza Place',
       'Bakery', 'Hyderabadi Restaurant', 'Tea Room', 'Shopping Mall',
       'Steakhouse', 'Eastern European Restaurant', 'Café',
       'Concert Hall', 'Chinese Restaurant', "Women's Store", 'Hotel',
       'Coffee Shop', 'Donut Shop', 'Bar', 'Sandwich Place', 'Wine Shop',
       'Market', 'Badminton Court', 'Clothing Store', 'Asian Restaurant',
       'Jewelry Store', 'Music Venue', 'South Indian Restaurant', 'Gym',
       'Trail', 'Candy Store', 'Event Space', 'Salad Place', 'Pool Hall',
       'Fried Chicken Joint', 'Breakfast Spot',
       'Vegetarian / Vegan Restaurant', 'Miscellaneous Shop',
       'Electronics Store', 'Travel & Transport', 'Metro Station',
       'Train Station'], dtype=object)

In [33]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# 6. Analyze Neighborhood

In [34]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3820, 198)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(55, 198)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.030000,0.010000,0.00,0.030000,0.010000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.010000,0.00,0.00,0.000000,0.020000,0.00,0.030000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.030000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.040000,0.000000,0.000000,0.010000,0.030000,0.230000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.00,0.0000

In [36]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

33

Create a new DataFrame for Shopping Mall data only

In [37]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [38]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,► Arekere‎ (5 F),0.01
1,► BEML‎ (5 F),0.01
2,"► Banashankari‎ (1 C, 4 F)",0.00
3,► Banaswadi‎ (2 F),0.01
4,"► Basavanagudi‎ (5 C, 11 F)",0.00


# 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Bangalore into 5 clusters.

In [39]:
# set number of clusters
kclusters = 4

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 1, 3, 3, 1, 1, 1, 3], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [41]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,► Arekere‎ (5 F),0.01,1
1,► BEML‎ (5 F),0.01,1
2,"► Banashankari‎ (1 C, 4 F)",0.00,3
3,► Banaswadi‎ (2 F),0.01,1
4,"► Basavanagudi‎ (5 C, 11 F)",0.00,3


In [42]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!


(55, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,► Arekere‎ (5 F),0.01,1,12.997982,77.610485
1,► BEML‎ (5 F),0.01,1,12.997982,77.610485
2,"► Banashankari‎ (1 C, 4 F)",0.00,3,12.939499,77.544498
3,► Banaswadi‎ (2 F),0.01,1,12.997819,77.610406
4,"► Basavanagudi‎ (5 C, 11 F)",0.00,3,12.939010,77.571360


In [43]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(55, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
27,► Kettohalli‎ (1 C),0.020000,0,12.957455,77.600908
25,"► Jayanagar, Bangalore‎ (1 C, 8 F)",0.020000,0,12.941649,77.596806
31,"► Krishnarajapura‎ (3 C, 3 F)",0.030303,0,13.000380,77.683780
32,"► Madiwala‎ (1 C, 6 F)",0.020000,0,12.957455,77.600908
35,► Majestic (Bangalore)‎ (1 C),0.020000,0,12.957455,77.600908
20,► Hosakote‎ (1 C),0.020000,0,12.957455,77.600908
40,► Nagarbhavi‎ (1 C),0.020000,0,12.957455,77.600908
10,"► Brigade Road, Bangalore‎ (3 C, 8 F)",0.020000,0,12.966273,77.606768
47,► Tippasandra‎ (1 C),0.020000,0,12.957455,77.600908
49,"► Ulsoor‎ (4 C, 3 F)",0.020000,0,12.989160,77.627980


Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# 8. Examine Clusters

Cluster 0

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
27,► Kettohalli‎ (1 C),0.020000,0,12.957455,77.600908
25,"► Jayanagar, Bangalore‎ (1 C, 8 F)",0.020000,0,12.941649,77.596806
31,"► Krishnarajapura‎ (3 C, 3 F)",0.030303,0,13.000380,77.683780
32,"► Madiwala‎ (1 C, 6 F)",0.020000,0,12.957455,77.600908
35,► Majestic (Bangalore)‎ (1 C),0.020000,0,12.957455,77.600908
20,► Hosakote‎ (1 C),0.020000,0,12.957455,77.600908
40,► Nagarbhavi‎ (1 C),0.020000,0,12.957455,77.600908
10,"► Brigade Road, Bangalore‎ (3 C, 8 F)",0.020000,0,12.966273,77.606768
47,► Tippasandra‎ (1 C),0.020000,0,12.957455,77.600908
49,"► Ulsoor‎ (4 C, 3 F)",0.020000,0,12.989160,77.627980


Cluster 1

In [47]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
39,"► Murugeshpalya‎ (4 C, 12 F)",0.012658,1,12.955580,77.653300
37,"► Marathahalli‎ (9 C, 1 P, 30 F)",0.013889,1,12.954670,77.707520
45,"► Shivajinagar‎ (1 C, 16 F)",0.010000,1,12.987190,77.604000
36,"► Malleswaram‎ (4 C, 2 F)",0.010526,1,12.994970,77.573450
34,► Mahadevapura‎ (2 C),0.010000,1,12.957986,77.600931
33,"► Magadi‎ (2 C, 10 F)",0.010000,1,12.958216,77.600357
48,"► UB City‎ (2 C, 16 F)",0.010000,1,12.958027,77.600996
26,► Jeevanbheemanagar‎ (3 F),0.010000,1,12.997855,77.610427
24,► Jakkur‎ (2 C),0.010000,1,12.957986,77.600931
41,► Rajajinagar‎ (3 C),0.013158,1,12.984582,77.555751


Cluster 2

In [48]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
51,"► Whitefield, Bangalore‎ (4 C, 1 P, 19 F)",0.046512,2,12.955743,77.720985
46,► Thubarahalli‎ (97 F),0.050000,2,12.953500,77.721130


Cluster 3

In [49]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
42,"► Rajarajeshwari Nagar, Bangalore‎ (1 C, 4 F)",0.0,3,12.940378,77.541510
21,"► Indiranagar‎ (4 C, 8 F)",0.0,3,12.978230,77.634290
44,► Sahakara Nagar‎ (1 C),0.0,3,13.060951,77.573992
4,"► Basavanagudi‎ (5 C, 11 F)",0.0,3,12.939010,77.571360
2,"► Banashankari‎ (1 C, 4 F)",0.0,3,12.939499,77.544498
52,"► Yelahanka‎ (1 C, 10 F)",0.0,3,13.099320,77.592600
43,"► Ramamurthy Nagar‎ (1 C, 20 F)",0.0,3,13.023780,77.677880
5,"► Begur, Bangalore‎ (1 C, 6 F)",0.0,3,12.882480,77.624730
38,► Mathikere‎ (1 C),0.0,3,13.030315,77.559694
11,"► Devanahalli‎ (4 C, 12 F)",0.0,3,13.249930,77.707920


# OBSERVATIONS:

Main focusing shopping Mall arround the Bangalore city with the highest number in cluster 1 and moderate number in cluster 2 and on cluster 2 is good compare to cluster 3 On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the over supply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 3 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.